# Задание 1

In [1]:
import re
import regex

"""
В regex поддерживаются частичные совпадения.
При использовании флага partial=True ищется соответствие префикса строки левой части регулярного выражения.
"""

re_pattern = re.compile(r'\w+енный')
regex_pattern = regex.compile(r'\w+енный')

print(re.search(re_pattern, 'построенный').group(0))
print(regex.search(regex_pattern, 'построенный').group(0))

print(re.search(re_pattern, 'построен'))
print(regex.search(regex_pattern, 'построен', partial=True).group(0))

# Здесь работает не так, как хотелось бы.
print(regex.search(regex_pattern, 'построен в 1921 году', partial=True).group(0))

построенный
построенный
None
построен
году


# Задание 2

In [2]:
with open('gramota.txt', 'r') as f:
    text = f.read().strip()

In [3]:
from string import punctuation
from gensim.utils import tokenize

text_tokenized = [x.lower() for x in tokenize(text)]
assert(all(not any(punct in token for punct in punctuation)
           for token in text_tokenized))
text_tokenized[:20]

['обособление',
 'обстоятельств',
 'обстоятельство',
 'второстепенный',
 'член',
 'предложения',
 'который',
 'обозначает',
 'признак',
 'действия',
 'или',
 'другого',
 'признака',
 'обстоятельства',
 'поясняют',
 'сказуемые',
 'или',
 'другие',
 'члены',
 'предложения']

In [4]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')
stems = {token: stemmer.stem(token) for token in text_tokenized}

In [5]:
from collections import defaultdict

stems2words = defaultdict(list)
for key, value in stems.items():
    stems2words[value].append(key)
{key: value for key, value in stems2words.items() if len(value) > 1}

{'обстоятельств': ['обстоятельств', 'обстоятельство', 'обстоятельства'],
 'член': ['член', 'члены'],
 'предложен': ['предложения', 'предложениях', 'предложении'],
 'котор': ['который', 'которые'],
 'признак': ['признак', 'признака'],
 'друг': ['другого', 'другие'],
 'сказуем': ['сказуемые', 'сказуемым'],
 'выделя': ['выделять', 'выделяется', 'выделяются'],
 'случа': ['случаях', 'случай', 'случаи'],
 'рассмотр': ['рассмотрим', 'рассмотрите'],
 'перв': ['первый', 'первом'],
 'выраж': ['выражены', 'выражено'],
 'нареч': ['наречием', 'наречия'],
 'деепричаст': ['деепричастием', 'деепричастий'],
 'оборот': ['оборотом', 'оборотами', 'оборот'],
 'улиц': ['улицу', 'улице', 'улицах'],
 'существительн': ['существительным', 'существительными'],
 'эт': ['этого', 'это'],
 'выражен': ['выражением', 'выраженные'],
 'он': ['он', 'оно', 'они', 'она'],
 'так': ['так', 'такие', 'такими'],
 'как': ['как', 'каких', 'какой', 'каким', 'каком'],
 'втор': ['втором', 'второй'],
 'не': ['не', 'нее'],
 'обособля'

К одинаковой основе были приведены разные слова: "выражением" и "выраженные", "не" и "нее", "следующие" и "следует" (хотя исторически это деепричастие, то есть форма этого глагола), "цели"/"целью" и "целый".

In [6]:
# Стеммер не справляется с супплетивными парадигмами, поскольку корни, а значит, и префиксы, разные.

print(stems['она'])
print(stems['нее'])

он
не


In [7]:
{key: value for key, value in stems.items() if len(key) > 4 and key == value}

{'обстоятельств': 'обстоятельств',
 'признак': 'признак',
 'могут': 'могут',
 'например': 'например',
 'дворник': 'дворник',
 'встает': 'встает',
 'может': 'может',
 'подряд': 'подряд',
 'сидел': 'сидел',
 'город': 'город',
 'вопрос': 'вопрос',
 'мороз': 'мороз',
 'оборот': 'оборот',
 'вдруг': 'вдруг',
 'имеет': 'имеет',
 'перед': 'перед'}

В случае словоформ "могут" и "может", "встает", "сидел", "имеет" — налицо ошибка.

# Задание 3

In [8]:
from nltk.corpus import stopwords
stops = stopwords.words('russian')

In [9]:
# Местоимения — частотная и слабо нагруженная семантически часть речи.

'почему' in stops

False

In [10]:
# То же самое в ещё большей степени относится к предлогам, даже к их усечённым вариантам.

'пред' in stops

False

In [11]:
# Здесь уже зависит от корпуса и контекста, но, например, в контексте
# гуманитарного или лингвистического дискурса слово "слово" — это стоп-слово.

'слово' in stops

False

In [12]:
# Разные формы одних и тех же слов включены в этот список, но почему это не сделано последовательно?

assert 'один' in stops
'одна' in stops

False

In [13]:
# "то" и "нибудь" есть в списке, но почему забыли "либо"?

assert 'то' in stops and 'нибудь' in stops
'либо' in stops

False

# Задание 4

In [14]:
with open('zhukov.txt', 'r') as f:
    text = f.read().strip()

## NLTK + regex + pymorphy2[fast]

In [15]:
from nltk import sent_tokenize

sentences = sent_tokenize(text)
sentences

['«Новая газета» опубликовала сокращенный вариант заключения экспертов Александра Коршикова и Анны Осокиной, которые анализировали видео из блога студента Высшей школы экономики Егора Жукова, обвиняемого в призывах к экстремизму в интернете (часть 2 статьи 280 УК).',
 'Эксперты нашли экстремизм в четырех видео на YouTube-канале Жукова.',
 'По их мнению, в видео «отчетливо прослеживается мотив политической ненависти или вражды к действующей власти».',
 'В заключении говорится, что Жуков приписывает действующей власти «враждебные действия», называет ее врагами, источником зла и вреда.',
 'Кроме того, студент призывает с этой властью бороться.',
 '«Автор описывает катастрофическое положение населения в России, как следствие целенаправленной политики действующей власти <...>.',
 'Основным идеалом, которого следует достичь, является "смена власти в России" для того, чтобы "Россия стала свободной"», — цитирует издание экспертов.',
 'В одном из видео, в котором Жуков размышлял об итогах митин

Разбиение на предложения безошибочно, придраться негде.

In [16]:
from nltk.tokenize import RegexpTokenizer

# возьмём регулярку из wordpunct_tokenize и слегка дополним:
# вся пунктуация будет отдельным токеном, кроме композитного символа "<...>"
tokenize = RegexpTokenizer(r'\w+|<...>|[«»\'",.:;!?\(\)-–—]|[^\w\s]+').tokenize
tokens = [tokenize(x) for x in sentences]
tokens[0]

['«',
 'Новая',
 'газета',
 '»',
 'опубликовала',
 'сокращенный',
 'вариант',
 'заключения',
 'экспертов',
 'Александра',
 'Коршикова',
 'и',
 'Анны',
 'Осокиной',
 ',',
 'которые',
 'анализировали',
 'видео',
 'из',
 'блога',
 'студента',
 'Высшей',
 'школы',
 'экономики',
 'Егора',
 'Жукова',
 ',',
 'обвиняемого',
 'в',
 'призывах',
 'к',
 'экстремизму',
 'в',
 'интернете',
 '(',
 'часть',
 '2',
 'статьи',
 '280',
 'УК',
 ')',
 '.']

In [17]:
from pymorphy2 import MorphAnalyzer

class Lemmatizer:
    def __init__(self):
        self.morph = MorphAnalyzer()
        self.cache = {}
    
    def lemmatize(self, token):
        if token in self.cache:
            return self.cache[token]
        norm = self.morph.parse(token)[0].normal_form
        self.cache[token] = norm
        return norm

In [18]:
lemmatize = Lemmatizer().lemmatize
lemmas = [[lemmatize(token) for token in sentence] for sentence in tokens]
lemmas[0]

['«',
 'новый',
 'газета',
 '»',
 'опубликовать',
 'сократить',
 'вариант',
 'заключение',
 'эксперт',
 'александр',
 'коршикова',
 'и',
 'анна',
 'осокин',
 ',',
 'который',
 'анализировать',
 'видео',
 'из',
 'блог',
 'студент',
 'высокий',
 'школа',
 'экономика',
 'егор',
 'жуков',
 ',',
 'обвинять',
 'в',
 'призыв',
 'к',
 'экстремизм',
 'в',
 'интернет',
 '(',
 'часть',
 '2',
 'статья',
 '280',
 'ук',
 ')',
 '.']

## pymystem3

In [19]:
from typing import List
def sent2tokens(sentence: str) -> List[str]:
    return re.findall(r'(?<={).*?(?=})', sentence, re.DOTALL)

# файл получен командой ./mystem -c -l -s -d <INPUT_PATH> <OUTPUT_PATH>
with open('zhukov_lemmas.txt', 'r') as f:
    lemmas = [sent2tokens(x) for x in f.read().strip().split('{\s}')]
lemmas[0]

['новый',
 'газета',
 'опубликовывать',
 'сокращать',
 'вариант',
 'заключение',
 'эксперт',
 'александр',
 'коршиков?',
 'и',
 'анна',
 'осокина',
 'который',
 'анализировать',
 'видео',
 'из',
 'блог',
 'студент',
 'высокий',
 'школа',
 'экономика',
 'егор',
 'жуков',
 'обвиняемый',
 'в',
 'призыв',
 'к',
 'экстремизм',
 'в',
 'интернет',
 'часть',
 'статья',
 'ук',
 'эксперт',
 'находить',
 'экстремизм',
 'в',
 'четыре',
 'видео',
 'на',
 'YouTube??',
 'канал',
 'жуков']

In [20]:
print(len(sentences), len(lemmas))

14 8


1. myStem выделяет меньше предложений, чем NLTK, сливая многие в одно. __1:0__
2. Регулярка токенизирует ровно так, как нам нужно, а её результат легко почистить (убрать токены-пунктуацию). myStem пропускает всю пунктуацию, не заключая её в фигурные скобки, поэтому, если она нам всё-таки нужна, придётся написать регулярку, чтобы извлекать её из выдачи myStem. Кроме того, чтобы распарсить его выход, уже пришлось написать одну регулярку. Единственный минус текущей регулярки на этом тексте — она разбивает на отдельные слова токены с дефисом ("кто-то"), но это легко поправить ещё одной группой через вертикальную черту. __2:0__
3. Качество лемматизации примерно одинаковое, хотя и здесь myStem слегка проигрывает, что довольно неожиданно, так как в нём реализовано контекстное снятие омонимии. Однако оно работает с ошибками, приводя глаголы к форме не того вида ("находить" вместо "найти" и пр.). Для словоформ "ее" и "того" приводятся неверные начальные формы ("ее" и "то" соответственно). Всех этих ошибок pymorphy2 не делает. __3:0__